In [3]:
# [adapted from an IPython log file]
# This notebook hopes to perform a basic test on the newly-made
# pathfinder environment.

import time; print(time.asctime())
'Fri Oct  6 00:58:37 2017'

import sys, imp
#sys.path.insert(0, '/home/usrnet/Software/installers/gym/')
sys.path.insert(0, '/home/usrnet/Work-Files/Path-Finder')
#sys.path.insert(0, '/home/usrnet/miniconda3/lib/python3.6/site-packages')
sys.path.insert(0, '/home/usrnet/Software/installers/gym/examples/agents')

import gym
from gym import envs, spaces
from gym.spaces import Discrete
from random_agent import *
from cem import *

import tensorflow as tf
import rl, keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

import cleaner

Sun Oct  8 22:11:10 2017
Done.


In [59]:
try: reload(cleaner)
except: 
    import imp, cleaner; imp.reload(cleaner)
#from cleaner import *
import cleaner; from cleaner import *

In [6]:
class MyEnv(gym.Env):
    def __init__(self, exp_env):
        self.action_space = Discrete(4)
        sz = exp_env.getGridSize()
        self.observation_space = spaces.Tuple((Discrete(sz[0]), \
                                Discrete(sz[1]), Discrete(4)))
        self.current_state = exp_env.get_random_starting_state()\
                                                    ['state']
        self.exp_env = exp_env        
        self.metadata = {'render.modes':['human','ansi','PRINT','NOPRINT']}
        self.reward_range = (0,1)
        
    def _reset(self): 
        self.current_state = self.exp_env.get_random_starting_state()['state']
        return self.current_state 
    
    def _render(self, mode=None, close=None): 
        p= print_state(self.current_state, \
                           'condensed', 'string_ret')
        if mode in ['human','PRINT']: 
            print(p)
            return p
        elif mode=='NOPRINT': 
            return p
        else: raise Exception(mode, 'render mode not defined')

    def sample(self): return random.choice(list(DVECS.keys()))

    def _step(self, actn):
        new_st, succ = self.exp_env.new_statem(\
            self.current_state, actn, valid_move_too=True)
        goalReached = self.exp_env.get_agent_loc(new_st) == \
                      self.exp_env.get_goal_loc(new_st)
        self.current_state = new_st
        return new_st, int(goalReached), (not succ) or goalReached, {}

In [4]:
my_env = MyEnv(cleaner.ExpAPI('tse2007', 'egocentric'))

model = Sequential()
model.add(Flatten(input_shape = (1,11,11,4) ))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(my_env.action_space.n, activation='linear'))
print(model.summary())
memory = SequentialMemory(limit=1000, window_length=1)
policy = EpsGreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions = my_env.action_space.n, memory=memory, nb_steps_warmup=10, enable_dueling_network=True, dueling_type='avg', target_model_update=1e-3, policy=policy)
dqn.compile(Adam(lr=1e-5), metrics=['mae'])

model2 = Sequential()
model2.add(Flatten(input_shape = (1,11,11,4) ))
model2.add(Dense(32))
model2.add(Activation('relu'))
model2.add(Dense(32))
model2.add(Activation('relu'))
model2.add(Dense(my_env.action_space.n, activation='linear'))
print(model2.summary())
memory = SequentialMemory(limit=10000, window_length=1)
policy = BoltzmannQPolicy()
dqn2 = DQNAgent(model=model2, nb_actions = my_env.action_space.n, memory=memory, nb_steps_warmup=300, enable_dueling_network=True, dueling_type='avg', target_model_update=1e-3, policy=policy)
dqn2.compile(Adam(lr=1e-2), metrics=['mae'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 484)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                15520     
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 132       
Total params: 15,652
Trainable params: 15,652
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
dqn.fit(my_env, nb_steps=16000, log_interval=2000, visualize=False, verbose=1);
dqn2.fit(my_env, nb_steps=16000, log_interval=2000, visualize=False, verbose=1)

Training for 16000 steps ...
Interval 1 (0 steps performed)
2000/2000 [==============================] - 9s - reward: 0.0095     
301 episodes - episode_reward: 0.063 [0.000, 1.000] - loss: 0.004 - mean_absolute_error: 0.045 - mean_q: 0.078

Interval 2 (2000 steps performed)
2000/2000 [==============================] - 9s - reward: 0.0080     
170 episodes - episode_reward: 0.094 [0.000, 1.000] - loss: 0.003 - mean_absolute_error: 0.058 - mean_q: 0.099

Interval 3 (4000 steps performed)
2000/2000 [==============================] - 9s - reward: 0.0070     
106 episodes - episode_reward: 0.132 [0.000, 1.000] - loss: 0.003 - mean_absolute_error: 0.055 - mean_q: 0.097

Interval 4 (6000 steps performed)
2000/2000 [==============================] - 9s - reward: 0.0090     
97 episodes - episode_reward: 0.186 [0.000, 1.000] - loss: 0.004 - mean_absolute_error: 0.079 - mean_q: 0.141

Interval 5 (8000 steps performed)
2000/2000 [==============================] - 9s - reward: 0.0055     
83 epis

Above was some tests. Now, attempt to build a model as similar to original pathfinder as possible.

In [60]:
ruru_env_ego = MyEnv(cleaner.ExpAPI('r-u-ru', 'egocentric'))
ruru_env_allo = MyEnv(cleaner.ExpAPI('r-u-ru', 'allocentric'))


-----
--  *
--I  
--   
-----

-----
- * -
-I  -
-   -
-----



In [67]:
for i in cardinals: # NSEW cardinals const global list of actions, inherited from environment.py
    ruru_env_ego.reset()
    ruru_env_ego.step(i)
    _=ruru_env_ego.render(mode='PRINT')

    ruru_env_allo.reset()
    ruru_env_allo.step(i)
    _=ruru_env_allo.render(mode='PRINT')

ego egocentric egocentric
-----
-----
- I -
-  *-
-   -

allo allocentric allocentric
-----
- I -
-  *-
-   -
-----

ego egocentric egocentric
- * -
-   -
- I -
-----
-----

allo allocentric allocentric
-----
-   -
-  *-
- I -
-----

ego egocentric egocentric
-----
  *--
  I--
   --
-----

allo allocentric allocentric
-----
- * -
-  I-
-   -
-----

ego egocentric egocentric
-----
-- * 
--I  
--   
-----

allo allocentric allocentric
-----
- * -
-I  -
-   -
-----

